# Node

In [1]:
# thanks to: 
# https://www.python-course.eu/python3_properties.php
# http://blog.lerner.co.il/python-attributes/
# https://www.programiz.com/python-programming/property

# this node can store any data types including itself.
class BaseNode(object):    
    def __init__(self, val=object(), max_child_count=0):
        self._dtype = type(val)
        self.value = val
        self._children = [None] * max_child_count
            
    @property
    def value(self):        
        return self._data
    
    @value.setter
    def value(self, val):        
        if not isinstance(val, self._dtype):
            raise TypeError('{} data-type cannot be stored in this Node({})'.format(type(val), self._dtype))
        self._data = val
        
    def child_count(self):
        return len(self._children) - self._children.count(None)
    
    def max_child_count(self):
        return len(self._children)
        
    def get_child(self, child_index):
        assert 0 <= child_index < len(self._children), 'Invalid child index'
        return self._children[child_index]
    
    def __getitem__(self, key):
        return self.get_child(key)
    
    def __setitem__(self, key, value):
        if value is None:
            self._children[key] = None
            return
        assert isinstance(value, type(self)), 'Children must be the same subclass as parent'
        assert 0 <= key < len(self._children), 'Invalid child index'        
        self._children[key] = value    
    
    def __repr__(self):
        return str({'child_count':self.child_count(), 'max_children':self.max_child_count(), 
                'value':self._data, 'dtype':self._dtype})
    
    def __str__(self):
        return '({}, {})'.format(self.value, [c is not None for c in self._children])


class Node(BaseNode):
    def __setitem__(self, key, value):
        if value is None:
            self._children[key] = None
            return
        assert isinstance(value, type(self)), 'Children must be the same subclass as parent'
        assert 0 <= key < len(self._children), 'Invalid child index'
        assert issubclass(value._dtype, self._dtype), 'dtypes do not match! (%s vs. %s)'\
                                                % (value._dtype, self._dtype)
        assert value.max_child_count() == self.max_child_count(), 'Maximum children must be %d'% self.max_child_count()
        self._children[key] = value

In [2]:
print(BaseNode(BaseNode(), 0))
print(repr(BaseNode(BaseNode(), 0)))
print('*'*30)

n = BaseNode(object(),2)
n.value = None
print(n)
print(repr(n))
print('*'*30)

n[0] = BaseNode('l',3)
n[0][0] = Node('ll')
n[0][1] = Node(777, 1)
n[0][2] = Node('lr')
print(n[0])
print(repr(n[0]))
print(n[0][0])
print(repr(n[0][0]))
print(n[0][1])
print(repr(n[0][1]))
print('*'*30)

n[1] = Node('r',2)
n[1][1] = Node('rr', 2)
print(n[1])
print(repr(n[1]))
print(n[1][1])
print(repr(n[1][1]))
print('*'*30)

((<object object at 0x7f89509d8cf0>, []), [])
{'child_count': 0, 'max_children': 0, 'value': {'child_count': 0, 'max_children': 0, 'value': <object object at 0x7f89509d8cf0>, 'dtype': <class 'object'>}, 'dtype': <class '__main__.BaseNode'>}
******************************
(None, [False, False])
{'child_count': 0, 'max_children': 2, 'value': None, 'dtype': <class 'object'>}
******************************
(l, [True, True, True])
{'child_count': 3, 'max_children': 3, 'value': 'l', 'dtype': <class 'str'>}
(ll, [])
{'child_count': 0, 'max_children': 0, 'value': 'll', 'dtype': <class 'str'>}
(777, [False])
{'child_count': 0, 'max_children': 1, 'value': 777, 'dtype': <class 'int'>}
******************************
(r, [False, True])
{'child_count': 1, 'max_children': 2, 'value': 'r', 'dtype': <class 'str'>}
(rr, [False, False])
{'child_count': 0, 'max_children': 2, 'value': 'rr', 'dtype': <class 'str'>}
******************************


# Tree

In [3]:
class BinaryTree(Node):
    def __init__(self, val):                
        Node.__init__(self, val, max_child_count=2)
                
    def __repr__(self):
        rep = {}        
        def subtree_repr(node, rep, level):
            if level not in rep.keys():
                rep[level] = list()
            if node is None:                
                rep[level].append('null')
                return
            
            rep[level].append(Node.__repr__(node))            
            for c in node._children:                
                subtree_repr(c, rep, level+1)
        subtree_repr(self, rep, 0)
        return 'tree: ' + repr(rep)
    
    def __str__(self):        
        def subtree_str(node):                                
            if node is None:                
                return '_'
            val_str = str(node.value)
            
            if len(node._children) == 0:
                return val_str
            
            val_str += '('
            for c in node._children:
                val_str += subtree_str(c) + ', '
            val_str = val_str[:-2] + ')'
            return val_str
        return subtree_str(self)

In [4]:
# example
BT = BinaryTree
root = BT('ROOT')
root[0] = BT('l')
root[1] = BT('r')
root[0][0] = BT('ll')
root[0][1] = BT('lr')
root[1][1] = BT('rr')
root[1][1][1] = BT('rrr')
print('*'*30)
print(str(root) + '\n')
print(repr(root))
print('*'*30)
print(root[1])
print(repr(root[1]))

******************************
ROOT(l(ll(_, _), lr(_, _)), r(_, rr(_, rrr(_, _))))

tree: {0: ["{'child_count': 2, 'max_children': 2, 'value': 'ROOT', 'dtype': <class 'str'>}"], 1: ["{'child_count': 2, 'max_children': 2, 'value': 'l', 'dtype': <class 'str'>}", "{'child_count': 1, 'max_children': 2, 'value': 'r', 'dtype': <class 'str'>}"], 2: ["{'child_count': 0, 'max_children': 2, 'value': 'll', 'dtype': <class 'str'>}", "{'child_count': 0, 'max_children': 2, 'value': 'lr', 'dtype': <class 'str'>}", 'null', "{'child_count': 1, 'max_children': 2, 'value': 'rr', 'dtype': <class 'str'>}"], 3: ['null', 'null', 'null', 'null', 'null', "{'child_count': 0, 'max_children': 2, 'value': 'rrr', 'dtype': <class 'str'>}"], 4: ['null', 'null']}
******************************
r(_, rr(_, rrr(_, _)))
tree: {0: ["{'child_count': 1, 'max_children': 2, 'value': 'r', 'dtype': <class 'str'>}"], 1: ['null', "{'child_count': 1, 'max_children': 2, 'value': 'rr', 'dtype': <class 'str'>}"], 2: ['null', "{'child_

# LinkedList

In [5]:
class LinkedList:
    def __init__(self):        
        self._first = None
        self._last = None
        self._size = 0
    
    def __len__(self):
        return self._size    
    
    def __getitem__(self, index):
        if not (isinstance(index, int) and 0 <= index < self._size):
            raise IndexError('Invalid index')
        n = self._first
        for i in range(index):
            n = n[0]
        return n.value
        
        
    def __setitem__(self, index, value):
        if not (isinstance(index, int) and 0 <= index < self._size):
            raise IndexError('Invalid index')
        n = self._first
        for i in range(index):
            n = n[0]
        n.value = value
    
    def append(self, value):
        self._size += 1
        if self._first is None:
            self._first = self._last = Node(value, 1)            
            return
        
        self._last[0] = Node(value, 1)
        self._last = self._last[0]        
        
    def pop(self):
        assert self._first is not None, 'Popping from empty list'
        self._size -= 1
        value = self._last.value
        if self._last == self._first:
            self._first = self._last = None            
            return value
        n = self._first
        while n[0] is not self._last:
            n = n[0]
        self._last = n
        self._last[0] = None
        return value
    
    def __repr__(self):
        rep = {}
        def list_repr(node, rep, level):
            if node is None:                                
                return
            rep[level] = node
            list_repr(node[0], rep, level+1)
        list_repr(self._first, rep, 1)
        return 'LinkedList(%s): %s' % (self._size, rep)
    
    def __str__(self):                
        def list_str(node):
            if node is None:                                
                return ''
            val = str(node.value) if type(node.value) != str else "'%s'" % node.value
            return "%s"% val + ', ' + list_str(node[0])
        return 'LinkedList: [' + list_str(self._first)[:-2] + ']'

In [6]:
ll = LinkedList()
ll.append('a')
ll.append('b')
ll.append('c')
ll.append('d')
ll[2] = 'ccc'
print(ll)
print(repr(ll))
print('*'*30)

ll.pop()
print(ll)
print(['a', 'b', 'ccc'])


LinkedList: ['a', 'b', 'ccc', 'd']
LinkedList(4): {1: {'child_count': 1, 'max_children': 1, 'value': 'a', 'dtype': <class 'str'>}, 2: {'child_count': 1, 'max_children': 1, 'value': 'b', 'dtype': <class 'str'>}, 3: {'child_count': 1, 'max_children': 1, 'value': 'ccc', 'dtype': <class 'str'>}, 4: {'child_count': 0, 'max_children': 1, 'value': 'd', 'dtype': <class 'str'>}}
******************************
LinkedList: ['a', 'b', 'ccc']
['a', 'b', 'ccc']


# Tips

## Decorators

In [7]:
# Thanks to https://www.programiz.com/python-programming/decorator

def star(func):
    def inner(*args, **kwargs):
        print('*' * 30)
        func(*args, **kwargs)
        print('*' * 30)
        return
    return inner

def nonnegative_integer(func):
    def inner(x):
        if not isinstance(x, int):
            raise TypeError('factorial is not defined for {}'.format(x))
        if x < 0:
            raise ValueError('factorial is not defined for negative numbers')
        return func(x)
    return inner

def printer(func):
    def inner(*args, **kwargs):
        print(func(*args, **kwargs))
    return inner
        

@nonnegative_integer
def factorial(n:int):
    if n==0 or n==1:
        return 1
    return n * factorial(n-1)

@star
@printer
@nonnegative_integer
def fact(n:int):
    if n==0 or n==1:
        return 1
    res=1
    for i in range(1,n+1):
        res *= i
    return res


def f(n:int):
    if n==0 or n==1:
        return 1
    res=1
    for i in range(1,n+1):
        res *= i
    return res
nondecorated_fact = star(printer(nonnegative_integer(f)))

print(factorial(5))
fact(5)
nondecorated_fact(5)

120
******************************
120
******************************
******************************
120
******************************
